In [ ]:
###FB
ADGEEK_FACEBOOK_API_ID = '958842090856883'
ADGEEK_FACEBOOK_APP_SECRET = 'a952f55afca38572cea2994d440d674b'
ADGEEK_FACEBOOK_ACCESS_TOKEN = 'EAANoD9I4obMBACygIE9jqmlaWeOW6tBma0oS6JbRpLgAvOYXpVi2XcXuasuwbBgqmaZBj5cP8MHE5WY2l9tAoi549eGZCP61mKr9BA8rZA6kxEW4ovX3KlbbrRGgt4RZC8MAi1UG0l0ZBUd0UBAhIPhzkZBi46ncuyCwkYPB7a6voVBZBTbEZAwH3azZA3Ph6g7aCOfxZCdDOp4AZDZD'
#ADGEEK_FACEBOOK_ACCESS_TOKEN = 'EAANoD9I4obMBALrHTgMWgRujnWcZA3ZB823phs6ynDDtQxnzIZASyRQZCHfr5soXBZA7NM9Dc4j9O8FtnlIzxiPCsYt4tmPQ6ZAT3yJLPuYQqjnWZBWX5dsOVzNhEqsHYj1jVJ3RAVVueW7RSxRDbNXKvK3W23dcAjNMjxIjQGIOgZDZD'

ACCOUNT_API_URL = 'https://mpc.adgeek.net/v2/accounts/'
ACCOUNT_TOEKN_API_URL = 'https://mpc.adgeek.net/v2/credentials/'
FACEBOOK_API_VERSION = "v4.0"
FACEBOOK_API_VERSION_URL = 'https://graph.facebook.com/{}/'.format(FACEBOOK_API_VERSION)

from facebook_business.session import FacebookSession
from facebook_business.api import FacebookAdsApi

def init_facebook_api(account_id = None):
    if not account_id:
        session = FacebookSession()
#         FacebookAdsApi(session, api_version="latest")
        FacebookAdsApi(session).init(ADGEEK_FACEBOOK_API_ID, ADGEEK_FACEBOOK_APP_SECRET, ADGEEK_FACEBOOK_ACCESS_TOKEN, api_version=FACEBOOK_API_VERSION)
        return ADGEEK_FACEBOOK_ACCESS_TOKEN
    
    try:
        query_id = get_queryid_by_accountid(account_id)
        token_dic = get_media_token_by_queryid(query_id)
        credential_id = token_dic['credential_id']
        credential_secret = token_dic['credential_secret']
        credential_token  = token_dic['credential_token']
        session = FacebookSession()
#         FacebookAdsApi(session, api_version="latest")
        
        FacebookAdsApi(session).init(credential_id, credential_secret, credential_token, api_version=FACEBOOK_API_VERSION)
        return credential_token

    except Exception as e:
        print('[init_facebook_api] error')
        print(e)
        session = FacebookSession()
#         FacebookAdsApi(session, api_version="latest")
        FacebookAdsApi(session).init(ADGEEK_FACEBOOK_API_ID, ADGEEK_FACEBOOK_APP_SECRET, ADGEEK_FACEBOOK_ACCESS_TOKEN, api_version=FACEBOOK_API_VERSION)
        return ADGEEK_FACEBOOK_ACCESS_TOKEN
##############################################################################

###GOOGLE
from googleads import adwords
from googleads import oauth2
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
adgeek_adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)

def init_google_api(account_id = None):
    if not account_id:
        return adgeek_adwords_client
    
    try:
        query_id = get_queryid_by_accountid(account_id)
        token_dic = get_media_token_by_queryid(query_id)
        credential_id = token_dic['credential_id']
        credential_secret = token_dic['credential_secret']
        credential_developer_token  = token_dic['credential_developer_token']
        credential_refresh_token = token_dic['credential_refresh_token']
        
        oauth2_client = oauth2.GoogleRefreshTokenClient(credential_id, credential_secret, credential_refresh_token)
        my_adwords_client = adwords.AdWordsClient(credential_developer_token, oauth2_client, client_customer_id=account_id)
        return my_adwords_client


    except Exception as e:
        print('[init_google_api] error')
        print(e)
        return adgeek_adwords_client


In [ ]:
import requests
import json 

def get_queryid_by_accountid(account_id):
    my_params = {'filter[account_id]': account_id}
    r = requests.get(ACCOUNT_API_URL, params = my_params)
    assert r.status_code == requests.codes.ok, "unexpected status code: {0}\n{1}".format(r.status_code, r.text)
    content = json.loads(r.text)
    data = content.get('_data')
    assert len(data) > 0, "account data empty: {}".format(content)
    first_data = data[0]
    query_id = first_data.get('credential_id')
    return query_id

def get_media_token_by_queryid(query_id):
    request_url = ACCOUNT_TOEKN_API_URL + str(query_id)
    r = requests.get(request_url)
    assert r.status_code == requests.codes.ok, "unexpected status code: {0}\n{1}".format(r.status_code, r.text)
    token_dic = {}
    content = json.loads(r.text)
    token_dic['credential_id'] = content.get('credential_id')#F #G
    token_dic['credential_secret'] = content.get('credential_secret')#F #G
    token_dic['credential_token'] = content.get('credential_token')#F
    token_dic['credential_developer_token'] = content.get('credential_developer_token')#G
    token_dic['credential_refresh_token'] = content.get('credential_refresh_token')#G
    token_dic['name'] = content.get('name')#G
#     print('[get_media_token_by_queryid] token_dic', token_dic)
    return token_dic

In [ ]:
def get_access_token_by_account(account_id):
    query_id = get_queryid_by_accountid(account_id)
    token_dic = get_media_token_by_queryid(query_id)
    credential_token  = token_dic['credential_token']
    return credential_token
    
def get_access_name_by_account(account_id):
    query_id = get_queryid_by_accountid(account_id)
    token_dic = get_media_token_by_queryid(query_id)
    credential_token  = token_dic['name']
    return credential_token
    

In [ ]:
if __name__=='__main__':
    init_facebook_api(341659359840575)
#     init_google_api(6714857152)

In [1]:
# !jupyter nbconvert --to script adgeek_permission.ipynb

[NbConvertApp] Converting notebook adgeek_permission.ipynb to script
[NbConvertApp] Writing 5453 bytes to adgeek_permission.py
